In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
#import re
#import numpy as np, pandas as pd
import os
#import datetime
import csv
import time
import random

In [ ]:
################################################################################
################################################################################
######################### 第一部分 数据整理 #########################

In [ ]:
############首先对Mapp.csv原文件数据去重，删除数据重复的项目######################
lis_a = []#创建空白列表，存储文件内容
csv_reader=csv.reader(open(r'C:\Users\rk\Desktop\Mapp.csv'))

In [ ]:
for element in csv_reader:
    lis_a.append(element)#逐行读入数据，写入列表

In [ ]:
lis_a

In [ ]:
####################运用此程序的前提是：先在原文件里做好分类汇总####################################
final_lis=[]
counter=0
while True:#为了是for循环中的起始值counter的改变，for进入循环后，起始值不再改变，
    for item_1 in lis_a[counter:]:
        diction=[]
        lis_c=[]
        initial_s=0
        lis_c.append(item_1[1])#将第一个期刊加入列表，在此处加入是防止仅出现一次而忽略
        for item_2 in lis_a[counter+1:]:
            if item_1[0]==item_2[0]:
                lis_c.append(item_2[1])#将相同的期刊加入列表
                initial_s+=1 #每找到一个相同的就把initial_s加一，     
        diction.append(item_1[0])
        diction.append(lis_c)
        final_lis.append(diction)
        break#为了使for循环中的起始值改变，循环一次后利用break强制退出for循环，利用while进入下一次循环
    counter=counter+initial_s+1
    if counter>=len(lis_a):#为了使while循环跳出,当大于列表长度的时候跳出循环
        break

In [ ]:
final_lis

In [ ]:
##############################################################################
##############################################################################
##################### 第二部分，下载文件 #####################################

In [ ]:
########################服务器崩溃后，从此处运行##########################################
browser = webdriver.Chrome('C:/Users/rk/Desktop/chromedriver.exe') #锁定浏览器


In [ ]:
browser.get("https://incites.thomsonreuters.com/#/explore/0/organization//")#访问网页

In [ ]:
"""
web of science 下载ISSN账号
Incite:602281532@qq.com
密码：rank2016!

"""

In [ ]:
def fid():
    '''查找保存按钮'''
    time_counter=0
    while True:
        try:
            browser.find_element_by_xpath('/html/body/div[1]/div/div[3]/div[1]/div/section/main/div[2]/div[3]/div[1]/div[1]/div[1]/a').click()
            send(items[0])
            time.sleep(1)
            browser.find_element_by_xpath('/html/body/div[1]/div/div[3]/div[1]/div/section/main/div[2]/div[3]/div[1]/div[1]/div[1]/div/ng-include/form/div[3]/button').click()
            break
        except:
            time.sleep(1)
            try:
                erro_tring=browser.find_element_by_xpath('/html/body/div[1]/div/div[3]/div[1]/div/section/main/div[2]/div[3]/div[2]').text
                if erro_tring== 'No available values':
                    break
            except:
                pass
            time_counter+=1
        if time_counter==5:
            break
            

In [ ]:
def send(item):
    '''输入需要保存的文件名'''
    while True:
        try:
            browser.find_element_by_xpath('//*[@id="fileName"]').clear()
            browser.find_element_by_xpath('//*[@id="fileName"]').send_keys(item)
        except:
            time.sleep(0.5)
        else:
            break

In [ ]:
counter=0

In [ ]:
item_xpath='//*[@id="s2id_autogen18"]'#这个地址很容易变，每次需要检查一下

In [ ]:
while True:
    for items in final_lis[counter:]:
        while True:#判断文本输入框是否为空，否则清空
            browser.find_element_by_xpath(item_xpath).send_keys(Keys.BACK_SPACE)#清空文本框的内容
            if len(browser.find_elements_by_xpath('//*[@id="s2id_sbjname"]/ul/li'))==1:#若元素只有一个，则跳出
                break
        time.sleep(0.5)        
        while True:#判断下拉框是否弹出，否则点击文本框
            try:
                browser.find_element_by_xpath('//*[@id="select2-drop"]/ul')#若不存在，则点击文本框 
            except:
                browser.find_element_by_xpath(item_xpath).click()
                time.sleep(2)
            else:
                break
            
        categorys=browser.find_elements_by_xpath('//*[@id="select2-drop"]/ul/li')#找到下拉列表里的所有学科
        small=[]
        for item in items[1]:
            small.append(item.lower())#将软科学科类别对应的学科全部小写，为了下面判断
        for i in range(len(categorys)):
            if (categorys[i].text).lower() in small:
                print(categorys[i].text)
                categorys[i].click()
                time.sleep(0.5)
                time.sleep(random.random()*3)
        browser.find_element_by_xpath('/html/body').click()#点击完学科后，在屏幕随便点一下，跳出文本框
        browser.find_element_by_xpath('/html/body/div[1]/div/div[3]/div[1]/div/section/main/div[2]/div[1]/div[5]/div/button').click()
        fid()    #查找下载按钮 
#         time.sleep(0.5)
#         send(items[0])#输入需要保存的文件
#         time.sleep(1)
#         browser.find_element_by_xpath('/html/body/div[1]/div/div[3]/div[1]/div/section/main/div[2]/div[3]/div[1]/div[1]/div[1]/div/ng-include/form/div[3]/button').click()
#         #print(item)
        counter+=1
        print(counter)
        break
    if counter==len(final_lis):
        break
#browser.close()
